# Imports

In [1]:
import numpy as np
import pandas as pd
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import wget
import tweepy
import json
import pprint as pp
from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
# This is needed to display the images.
# %matplotlib inline

# %config InlineBackend.figure_format = 'retina'

from utils import label_map_util

from utils import visualization_utils as vis_util

/Users/brianosgood/Documents/PyFilter/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/brianosgood/.virtualenvs/huracan/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/brianosgood/.virtualenvs/huracan/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/brianosgood/.virtualenvs/huracan/lib/python3.6/site-packages/ipyker

In [6]:
def JsonParser(data):
    import json
    import ast
    if data == "":
        pass
    else:
        data = json.dumps(ast.literal_eval(data))
        j1 = json.loads(data)
    return j1

In [7]:
def column_creator(col):
    for _ in col:
        tweets[_] = ""
        if _ == 'Hashtags':
            for i,v in tweets.iterrows():
                hashes = []
                for tag in v['Entities'].get('hashtags'):
                    hashes.append(tag['text'])
                    tweets.at[i,_] = hashes
                if 'hashtags' not in v['Entities']:
                    hashes.append('No ' + _ )
                tweets.at[i,_] = hashes
        elif _ == 'urls':
            for i,v in tweets.iterrows():
                for tag in v['Entities'].get(_):
                    tweets.at[i,'urls'] = tag['url']
                if _ not in v['Entities']:
                    tweets.at[i,'urls'] = 'No '+ _
        elif _ == 'media_url':
            for i,v in tweets.iterrows():
                if 'media'in v['Entities'].keys():
                    tweets.at[i,_] = (v['Entities']['media'][0][_])
                elif 'entities' in v['Extended Entities'].keys():
                    if 'media' in v['Extended Entities']['entities'].keys():
                        tweets.at[i,_] = (v['Extended Entities']['entities']['media'][0][_])
                    elif 'media' not in v['Extended Entities']['entities'].keys():
                        tweets.at[i,_] = 'No ' + _
                elif 'media' not in v['Entities'].keys():
                    tweets.at[i,_] = 'No ' + _
    pattern = '.+\/(\w.+)'
    try:
        tweets['image_name'] = tweets['media_url'].str.extract(pattern)
    except:
        pass

In [8]:
tweets = pd.read_csv("../Twitter/huracan.csv", converters={"Entities":JsonParser,"Extended Entities":JsonParser}) #read in our csv from the twitter connection


col = ['Hashtags','urls','media_url']

column_creator(col)

In [9]:
def get_media():
    import wget
    try:
        for url in tweets['media_url']:
            if ".jpg" in url:
                wget.download(url, out=f'../images/')
    except:
        return 'Something Went Wrong!'

In [10]:
get_media()

# Model preparation 

In [21]:
# What model to download.
MODEL_NAME = 'lambo_detection_graph/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + 'frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'lambo_detection.pbtxt')

NUM_CLASSES = 4

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

path = '../images/'
photos = os.listdir(path=path)

PATH_TO_TEST_IMAGES_DIR = path
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i[:-4])) for i in photos if i[-4:] == '.jpg']

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 12)

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

to_tweet = {'to_tweet':[],
            'to_tweet_file':[],
            'image_score':[],
           }
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        object_dict = {}
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, detection_graph)
            # Visualization of the results of a detection.
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            threshold = 0.1

            (boxes, scores, classes, num) = sess.run(
                                                    [detection_boxes, 
                                                     detection_scores, 
                                                     detection_classes, 
                                                     num_detections],
                                                     feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(image=image_np,
                                                               boxes = output_dict['detection_boxes'],
                                                               classes = output_dict['detection_classes'],
                                                               scores = output_dict['detection_scores'],
                                                               category_index = category_index,
                                                               instance_masks = output_dict.get('detection_masks'),
                                                               min_score_thresh = threshold,
                                                               use_normalized_coordinates=True,
                                                               line_thickness=8)         
            for index, value in enumerate(classes[0]):
                if scores[0, index] > threshold and (category_index.get(value)).get('name') == 'huracan':
                    to_tweet['to_tweet'].append(1)
                    to_tweet['to_tweet_file'].append(image_path[14:])
                    to_tweet['image_score'].append(scores[0][0])
                    plt.figure(figsize=IMAGE_SIZE)
                    plt.imshow(image_np)


In [19]:
scores[0][0]

0.7579023

In [12]:
photos

['00000013.jpg',
 'Dh4SCf9X4AEe7CU (1).jpg',
 '.DS_Store',
 '00000017.jpg',
 '00000015.jpg',
 '00000014.jpg',
 '00000018.jpg',
 'Dh4SCf9X4AEe7CU.jpg']

In [13]:
to_tweet = pd.DataFrame(data=to_tweet)

In [14]:
tweets['post'] = tweets['image_name'].isin(to_tweet['to_tweet_file'])

In [28]:
for index,image in tweets.iterrows():
    if image['post'] == True:
        status = "beep boop I'm an Image recognition Bot #Huracan #Lamborghini "
        tweet_it = api.update_with_media(filename="./test_images/"+str(image['image_name']),status= status)
        
        

In [ ]:
tweets.to_csv("../")

In [16]:
newest_post = max(tweets['Date'])

In [17]:
newest_post

'2018-07-11 23:04:00'

In [29]:
%%bash

rm ./test_images/*